In [1]:
import sys
import os
import numpy as np
import pandas as pd
import torch
from PIL import Image

import ipywidgets as widgets
from io import BytesIO

from ganpreffinder import GANPrefFinder
from jupyterGUI import GUIses
from jupyterComponentsExprGUI import GUICompExprSes

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


### Description:

Possible datasets: ffhq, car, cat, church, horse, bedrooms, kitchen, places

---

In [2]:
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda:0"

print("Device used:", DEVICE)

Device used: cpu


In [3]:
# (idx, start, end, strength, invert)
# 'Redness':          (22,  9, 11,   -8, False),
# 'Horizontal flip':  ( 0,  0,  5,  2.0, True),
# 'Add grass':        (41,  9, 11,  -18, False),
# 'Blocky shape':     (16,  3,  6,   25, False),

component_layers_dict = {
    0: (0, 5),
    16: (3, 6),
    22: (9, 11),
    41: (9, 11)
}
component_indices_to_use = [0,1,3,4,6,8]

In [4]:
gpf = GANPrefFinder(class_name           = "car",
                    gan_sample_seed      = 0,
                    n_comp_in_use        = None,
                    comp_to_use          = component_indices_to_use,
                    # comp_layers_dict   = component_layers_dict,
                    adaptive_components  = component_indices_to_use,
                    strength_left_bound  = -10,
                    strength_right_bound = 10,
                    acquisition_strategy = 'EI-EXT',
                    device=DEVICE)

In [5]:
# gui_comp_ses = GUICompExprSes(gpf)
# gui_comp_ses.run()

In [6]:
guises = GUIses(gpf)
guises.run()

In [7]:
# CONTINUE

---